In [10]:
from googleapiclient.discovery import build
from youtube_api import YouTubeDataAPI
import json
import time

In [19]:

# get my_api from API_Key.py
my_api = 'API_KEY'

In [20]:
# apikey 인식
youtube = build('youtube', 'v3', developerKey = my_api)
yt = YouTubeDataAPI(my_api)

In [21]:
# 함수
def videodata (id_list):
    title = []
    category = []
    desc = []
    view = []
    like = []
    dislike = []
    comment = []
    tag = []
    for data in id_list:
        datas = yt.get_video_metadata(data)
        title.append(datas['video_title'])
        category.append(datas['video_category'])
        desc.append(datas['video_description'].strip())
        view.append(datas['video_view_count'])
        like.append(datas['video_like_count'])
        dislike.append(datas['video_dislike_count'])
        comment.append(datas['video_comment_count'])
        tag.append(datas['video_tags'])
    df = pd.DataFrame({
    'category':category,
    'description':desc,
    'view_count':view,
    'like_count':like,
    'dislike_count':dislike,
    'com_count':comment,
    'tags':tag
    })
    df.fillna(0,inplace = True)
    return df

In [93]:
# 테스트
res1 = youtube.search().list(
    q = 'vitamin', 
    maxResults = 5,
    order = 'viewCount',
    publishedBefore = '2020-01-01T00:00:00Z',
    publishedAfter = '2019-01-01T00:00:00Z',
    regionCode = 'US',
    relevanceLanguage = 'en',
    part = 'snippet', 
    type = 'video'

).execute()

In [4]:
sup_list = ['Milk thistle', 'Creatine', 'Curcumin', 'Omega 3',
'Testosterone Boosters','Vitamin C','Vitamin D','CoQ10',
'Multivitamin','Zinc','Probiotic','B-complex','Calcium',
'Melatonin','Vitamin K','Collagen','Coconut oil','Lutein','Nicotine',
'Ashwagandha','Maca','L-carnitine','Kombucha','Krill oil',
'Hibiscus', 'Amino acid','Vitamin A','Vitamin B']

In [14]:
k_sup_list = ['밀크씨슬', '크레아틴','커큐민', '오메가3', '정력제', '비타민 C', '비타민 D', '코큐텐',
              '멀티 비타민', '아연', '유산균', 'B 복합체', '칼슘', '멜라토닌', '비타민 K', '콜라겐',
            '코코넛 오일', '루테인', '니코틴', '아쉬와간다', '마카', 'L-카르니틴', '콤부차', '크릴오일',
            '히비스커스', '아미노산', '비타민 A', '비타민 B']

In [22]:
#acquiring videoid
videoid = []
pdata = []
name = []
nextpagetoken = []
# insert name of supplements
for data in k_sup_list:
    res = youtube.search().list(q = data, maxResults = 50,
    order = 'viewCount', publishedBefore = '2018-01-01T00:00:00Z',
    publishedAfter = '2017-01-01T00:00:00Z',regionCode = 'KR',
    relevanceLanguage = 'ko', part = 'snippet', type = 'video').execute()
    
    for idx in range(len(res['items'])):
        videoid.append(res['items'][idx]['id']['videoId'])
        pdata.append(res['items'][idx]['snippet']['publishedAt'])
        name.append(data)
    
    res1 = youtube.search().list(q = data, maxResults = 50,
    order = 'viewCount', publishedBefore = '2018-01-01T00:00:00Z',
    publishedAfter = '2017-01-01T00:00:00Z',regionCode = 'KR',
    relevanceLanguage = 'ko', part = 'snippet', 
    type = 'video',pageToken = res['nextPageToken']).execute()
    
    for idx in range(len(res1['items'])):
        videoid.append(res1['items'][idx]['id']['videoId'])
        pdata.append(res1['items'][idx]['snippet']['publishedAt'])
        name.append(data)
        nextpagetoken.append(res1['nextPageToken'])
    
len(videoid),len(pdata)

(1308, 1308)

In [23]:
#making df
sup_df = videodata(videoid)
sup_df['videoid'] = videoid
sup_df['sup_name'] = name

In [24]:
#save
sup_df.to_csv('2017supplements_KOR.csv',sep=',',encoding='UTF-8', index=False)